In [1]:
import sys
sys.path.append('..')

In [15]:
import tensorflow as tf
from keras_tweaks import (
    get_sparsity_pattern,
    shortest_path_to_origin,
    feedback_signal_patterns
)
import random

In [3]:
dim = 10

# Block Matrix

In [16]:
mat_pattern = get_sparsity_pattern('block', dim, block_sizes=[4, 1, 2])
mat_values = range(1, len(mat_pattern)+1)
print(len(mat_pattern))

mat = tf.sparse.SparseTensor(
    dense_shape=(dim, dim),
    indices=mat_pattern,
    values=mat_values)

print(tf.sparse.to_dense(mat))

30
tf.Tensor(
[[ 1  2  3  4  0  0  0  0  0  0]
 [ 5  6  7  8  0  0  0  0  0  0]
 [ 9 10 11 12  0  0  0  0  0  0]
 [13 14 15 16  0  0  0  0  0  0]
 [ 0  0  0  0 17  0  0  0  0  0]
 [ 0  0  0  0  0 18 19  0  0  0]
 [ 0  0  0  0  0 20 21  0  0  0]
 [ 0  0  0  0  0  0  0 22 23 24]
 [ 0  0  0  0  0  0  0 25 26 27]
 [ 0  0  0  0  0  0  0 28 29 30]], shape=(10, 10), dtype=int32)


## always immediate feedback
In recurrent kernel, a block matrix pattern consist basically of several smaller dense matrices.
Thus, information from state $i$ bounces back from state $j$ immediatly after one recursion step.

In [5]:
pathlens = shortest_path_to_origin(W=mat, max_recur=dim)
pathlens

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [6]:
signals = feedback_signal_patterns(W=mat, max_recur=dim)
signals

array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=int8)

# Triangular Matrix

## triu

In [7]:
mat_pattern = get_sparsity_pattern('triu', n=dim, k=-1)
print(len(mat_pattern))

mat = tf.sparse.SparseTensor(
    dense_shape=(dim, dim),
    values=range(1, len(mat_pattern)+1),
    indices=mat_pattern)

print(tf.sparse.to_dense(mat))

45
tf.Tensor(
[[ 0  1  2  3  4  5  6  7  8  9]
 [ 0  0 10 11 12 13 14 15 16 17]
 [ 0  0  0 18 19 20 21 22 23 24]
 [ 0  0  0  0 25 26 27 28 29 30]
 [ 0  0  0  0  0 31 32 33 34 35]
 [ 0  0  0  0  0  0 36 37 38 39]
 [ 0  0  0  0  0  0  0 40 41 42]
 [ 0  0  0  0  0  0  0  0 43 44]
 [ 0  0  0  0  0  0  0  0  0 45]
 [ 0  0  0  0  0  0  0  0  0  0]], shape=(10, 10), dtype=int32)


## tril

In [8]:
# tril is the transposed of triu
mat_pattern = tf.keras.backend.reverse(mat_pattern, axes=1)
print(len(mat_pattern))

mat = tf.sparse.reorder(tf.sparse.SparseTensor(
    dense_shape=(dim, dim),
    values=range(1, len(mat_pattern)+1),
    indices=mat_pattern))

print(tf.sparse.to_dense(mat))

45
tf.Tensor(
[[ 0  0  0  0  0  0  0  0  0  0]
 [ 1  0  0  0  0  0  0  0  0  0]
 [ 2 10  0  0  0  0  0  0  0  0]
 [ 3 11 18  0  0  0  0  0  0  0]
 [ 4 12 19 25  0  0  0  0  0  0]
 [ 5 13 20 26 31  0  0  0  0  0]
 [ 6 14 21 27 32 36  0  0  0  0]
 [ 7 15 22 28 33 37 40  0  0  0]
 [ 8 16 23 29 34 38 41 43  0  0]
 [ 9 17 24 30 35 39 42 44 45  0]], shape=(10, 10), dtype=int32)


## no feedback connections
There are no feedback connections to the origin in triangular recurrent kernels.

In [9]:
pathlens = shortest_path_to_origin(W=mat, max_recur=dim)
print(pathlens)

[None, None, None, None, None, None, None, None, None, None]


In [10]:
signals = feedback_signal_patterns(W=mat, max_recur=dim)
print(signals)

[[0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]]


# Circular pattern

## The smallest recurrent pattern
Information from input state $i$ return to its origin after $t=dim$ steps.

In [11]:
mat_pattern = get_sparsity_pattern('circle', n=dim, offsets=[1])
mat_values = range(1, len(mat_pattern)+1)
print(len(mat_pattern))

mat = tf.sparse.SparseTensor(
    dense_shape=(dim, dim),
    indices=mat_pattern,
    values=mat_values)

print(tf.sparse.to_dense(mat), "\n")

pathlens = shortest_path_to_origin(W=mat, max_recur=dim)
print(pathlens, "\n")

signals = feedback_signal_patterns(W=mat, max_recur=dim)
print(signals)

10
tf.Tensor(
[[ 0  0  0  0  0  0  0  0  0  1]
 [ 2  0  0  0  0  0  0  0  0  0]
 [ 0  3  0  0  0  0  0  0  0  0]
 [ 0  0  4  0  0  0  0  0  0  0]
 [ 0  0  0  5  0  0  0  0  0  0]
 [ 0  0  0  0  6  0  0  0  0  0]
 [ 0  0  0  0  0  7  0  0  0  0]
 [ 0  0  0  0  0  0  8  0  0  0]
 [ 0  0  0  0  0  0  0  9  0  0]
 [ 0  0  0  0  0  0  0  0 10  0]], shape=(10, 10), dtype=int32) 

[10, 10, 10, 10, 10, 10, 10, 10, 10, 10] 

[[0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 1]]


## Similar types of processes
The circular pattern will simulate similar type of processes.

In [12]:
mat_pattern = get_sparsity_pattern('circle', n=dim, offsets=[1, 3])
mat_values = range(1, len(mat_pattern)+1)
print(len(mat_pattern))

mat = tf.sparse.SparseTensor(
    dense_shape=(dim, dim),
    indices=mat_pattern,
    values=mat_values)

print(tf.sparse.to_dense(mat), "\n")

pathlens = shortest_path_to_origin(W=mat, max_recur=dim)
print(pathlens, "\n")

signals = feedback_signal_patterns(W=mat, max_recur=dim)
print(signals)

20
tf.Tensor(
[[ 0  0  0  0  0  0  0  1  0  2]
 [ 3  0  0  0  0  0  0  0  4  0]
 [ 0  5  0  0  0  0  0  0  0  6]
 [ 7  0  8  0  0  0  0  0  0  0]
 [ 0  9  0 10  0  0  0  0  0  0]
 [ 0  0 11  0 12  0  0  0  0  0]
 [ 0  0  0 13  0 14  0  0  0  0]
 [ 0  0  0  0 15  0 16  0  0  0]
 [ 0  0  0  0  0 17  0 18  0  0]
 [ 0  0  0  0  0  0 19  0 20  0]], shape=(10, 10), dtype=int32) 

[4, 4, 4, 4, 4, 4, 4, 4, 4, 4] 

[[0 0 0 1 0 1 0 1 0 1]
 [0 0 0 1 0 1 0 1 0 1]
 [0 0 0 1 0 1 0 1 0 1]
 [0 0 0 1 0 1 0 1 0 1]
 [0 0 0 1 0 1 0 1 0 1]
 [0 0 0 1 0 1 0 1 0 1]
 [0 0 0 1 0 1 0 1 0 1]
 [0 0 0 1 0 1 0 1 0 1]
 [0 0 0 1 0 1 0 1 0 1]
 [0 0 0 1 0 1 0 1 0 1]]


# Random sparsity pattern
The shortest paths back to the origin states is more diverse.

In [13]:
random.seed(23)
mat_pattern = get_sparsity_pattern('random', r=dim, c=dim, pct=0.20)
mat_values = range(1, len(mat_pattern)+1)
print(len(mat_pattern))

mat = tf.sparse.SparseTensor(
    dense_shape=(dim, dim),
    indices=mat_pattern,
    values=mat_values)

print(tf.sparse.to_dense(mat), "\n")

pathlens = shortest_path_to_origin(W=mat, max_recur=dim)
print(pathlens, "\n")

signals = feedback_signal_patterns(W=mat, max_recur=dim)
print(signals)

20
tf.Tensor(
[[ 1  0  0  0  0  0  0  2  0  0]
 [ 0  0  0  0  3  0  0  0  4  0]
 [ 0  5  0  6  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  7  8]
 [ 9  0  0  0  0  0  0  0  0 10]
 [ 0  0  0  0  0  0 11 12  0  0]
 [ 0 13  0  0  0 14  0  0  0  0]
 [ 0  0 15  0 16  0  0  0  0  0]
 [ 0  0 17 18  0  0  0  0  0  0]
 [ 0  0  0  0  0 19 20  0  0  0]], shape=(10, 10), dtype=int32) 

[1, 3, 3, 2, 3, 2, 2, 3, 2, 4] 

[[1 1 1 1 1 1 1 1 1 1]
 [0 0 1 1 1 1 1 1 1 1]
 [0 0 1 0 1 1 1 1 1 1]
 [0 1 1 1 1 1 1 1 1 1]
 [0 0 1 1 1 1 1 1 1 1]
 [0 1 0 1 1 1 1 1 1 1]
 [0 1 0 1 1 1 1 1 1 1]
 [0 0 1 1 1 1 1 1 1 1]
 [0 1 1 1 1 1 1 1 1 1]
 [0 0 0 1 1 1 1 1 1 1]]


# Random without diagonal elements

In [14]:
random.seed(0)
mat_pattern = get_sparsity_pattern('random2', n=dim, pct=0.20)
mat_values = range(1, len(mat_pattern)+1)
print(len(mat_pattern))

mat = tf.sparse.SparseTensor(
    dense_shape=(dim, dim),
    indices=mat_pattern,
    values=mat_values)

print(tf.sparse.to_dense(mat), "\n")

pathlens = shortest_path_to_origin(W=mat, max_recur=dim)
print(pathlens, "\n")

signals = feedback_signal_patterns(W=mat, max_recur=dim)
print(signals)

20
tf.Tensor(
[[ 0  0  1  0  0  0  2  0  0  0]
 [ 3  0  0  0  0  0  0  0  4  0]
 [ 0  0  0  0  5  0  0  6  0  0]
 [ 0  0  7  0  0  8  0  0  0  0]
 [ 0  0  0  9  0  0  0  0  0 10]
 [ 0 11  0 12  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0 13 14]
 [ 0 15  0  0  0  0 16  0  0  0]
 [ 0  0  0  0 17  0  0 18  0  0]
 [19  0  0  0  0 20  0  0  0  0]], shape=(10, 10), dtype=int32) 

[3, 3, 3, 2, 3, 2, 3, 3, 3, 3] 

[[0 0 1 1 1 1 1 1 1 1]
 [0 0 1 1 1 1 1 1 1 1]
 [0 0 1 1 1 1 1 1 1 1]
 [0 1 1 1 1 1 1 1 1 1]
 [0 0 1 1 1 1 1 1 1 1]
 [0 1 0 1 1 1 1 1 1 1]
 [0 0 1 0 1 1 1 1 1 1]
 [0 0 1 1 1 1 1 1 1 1]
 [0 0 1 0 1 1 1 1 1 1]
 [0 0 1 1 1 1 1 1 1 1]]
